In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import datetime
import scipy.stats as stats

#graphing
import matplotlib.pyplot as plt
#stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

#import testing
import sys
sys.path.append("../")
import selection_tests

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
#need to replicate table 3?
school_data = pd.read_stata('aerdata_colombia2.dta')
#keep if checkid==1 & age>=9 & age<=25
school_data = school_data[(school_data['checkid']==1) &
                          (school_data['age']>=9) & (school_data['age']<=25) ]
print(school_data.columns)

Index(['vouch0', 'sex_name', 'match_i', 'match_ic', 'match_i7', 'age', 'phone',
       'checkid', 'math', 'read', 'match_ic7', 'readcens1', 'readcens10',
       'mathcens1', 'mathcens10'],
      dtype='object')


In [6]:
# Fit the model with ols = True
x_names =['vouch0', 'age', 'sex_name'] 
y_name = 'readcens10'
#different datasets
cutoff = stats.mode(school_data[y_name])[0][0]
print(cutoff)
school_nozeros = school_data[ (school_data[y_name]> cutoff) ]
print(school_data.shape,school_nozeros.shape)


#setup x/y for missing data models
x = school_nozeros[x_names].copy()
y = school_nozeros[y_name]

ols_model = sm.OLS(y,x).fit()
print(ols_model.summary())
#print(ols_model.summary().as_latex())

#setup x/y for missing data models
x = school_data[x_names].copy()
y = school_data[y_name]

40.0
(3542, 15) (1091, 15)
                                 OLS Regression Results                                
Dep. Variable:             readcens10   R-squared (uncentered):                   0.986
Model:                            OLS   Adj. R-squared (uncentered):              0.986
Method:                 Least Squares   F-statistic:                          2.626e+04
Date:                Wed, 23 Jun 2021   Prob (F-statistic):                        0.00
Time:                        19:19:37   Log-Likelihood:                         -3446.3
No. Observations:                1091   AIC:                                      6899.
Df Residuals:                    1088   BIC:                                      6914.
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------

In [41]:
tobit_model = selection_tests.Tobit(y,x, cc=True).fit()
print(tobit_model.summary(title='OLS CC'))
print('\n')
#why is there randomly a missing observation?
#print(tobit_model.summary(title='OLS CC').as_latex())

../selection_tests.py:73: RuntimeWarning: invalid value encountered in greater
  #start with OLS as a guess
../selection_tests.py:83: RuntimeWarning: invalid value encountered in less_equal
  x = self.exog


                                    OLS CC                                    
Dep. Variable:             readcens10   Log-Likelihood:                    nan
Model:                          Tobit   AIC:                               nan
Method:            Maximum Likelihood   BIC:                               nan
Date:                Wed, 23 Jun 2021                                         
Time:                        19:01:22                                         
No. Observations:                3542                                         
Df Residuals:                    3539                                         
Df Model:                           2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
vouch0         0.9733        nan        nan        nan         nan         nan
age            3.9932        nan        nan        n

/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [42]:
# Fit the model with ols = False
tobit_model = selection_tests.Tobit(y,x).fit()

print(tobit_model.summary())
print('\n')
#print(tobit_model.summary().as_latex())

                                Tobit Results                                 
Dep. Variable:             readcens10   Log-Likelihood:                    nan
Model:                          Tobit   AIC:                               nan
Method:            Maximum Likelihood   BIC:                               nan
Date:                Wed, 23 Jun 2021                                         
Time:                        19:01:24                                         
No. Observations:                3542                                         
Df Residuals:                    3539                                         
Df Model:                           2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
vouch0         0.9733        nan        nan        nan         nan         nan
age            3.9932        nan        nan        n

/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [43]:
# Fit the regular test model
regular_test_model = selection_tests.regular_test(y,x,selection_tests.setup_test)
regular_test_model

/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


regular: test, llr, omega ----
nan nan nan
---- 


/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


(0, nan)

In [32]:
# Fit the bootstrap test model
bootstrap_test_model = selection_tests.bootstrap_test(y,x,selection_tests.setup_test, trials=100)
bootstrap_test_model

32.0
[ 1.10327664  3.87856448  1.10563608 42.24087001]


/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


32.0
[ 1.10327664  3.87856448  1.10563608 42.24087001]


/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


ValueError: array must not contain infs or NaNs